In [1]:
# Enter project folder using cd command or simple put the notebook under the project folder
%cd /content/drive/MyDrive/diffusion/stable-diffusion

/content/drive/MyDrive/diffusion/stable-diffusion


In [2]:
!pip install torch torchvision
!pip install transformers==4.19.2 diffusers invisible-watermark
!pip install -e .
!pip install diffusers -q
!pip install einops kornia
!pip install transformers -q -UU ftfy gradio 
!pip install taming-transformers-rom1504
!pip install git+https://github.com/openai/CLIP.git -q

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.19.2-py3-none-any.whl (4.2 MB)
  Using cached protobuf-3.20.3-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.0 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.1
    Uninstalling protobuf-3.20.1:
      Successfully uninstalled protobuf-3.20.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.25.1
    Uninstalling transformers-4.25.1:
      Successfully uninstalled transformers-4.25.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorboardx 2.5.1 requires pr

In [3]:
#@title Path Setting { display-mode: "form" }
IMAGE = 'testpic5.png' #@param{'type':'string'}
FOLDER = 'outputs3' #@param{'type':'string'}
HISTORY = 'history.txt' #@param{'type':'string'}
EXISTING_MASK = "mask_ear.png" #@param{'type':'string'}
CONFIG = 'models/ldm/anything/config.yaml' #@param{'type':'string'}
CHECKPOINT = 'models/ldm/anything/Anything-V3.0-pruned-fp32.ckpt' #@param{'type':'string'}
VAECKPOINT = 'models/ldm/anything/Anything-V3.0.vae.pt' #@param{'type':'string'}
CLIPSEG = 'models/clipseg_weights/rd64-uni-refined.pth' #@param{'type':'string'}


In [4]:
#@title Models Setting { display-mode: "form" }
HEIGHT = 640        #@param{'type':'number'}
WIDTH = 640         #@param{'type':'number'}

PROMPTS = "" #@param{'type':'string'}
PROMPTS=PROMPTS.split('\",')

NEGATIVES = "nsfw, lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry, bad feet" #@param{'type':'string'}
NEGATIVES=NEGATIVES.split('\",')

# Mask Settings
MASK = 'hair'      #@param{'type':'string'}
MASK = MASK.split('\",')
USE_EXISTING_MASK = True #@param{'type':'boolean'}
if USE_EXISTING_MASK:
  MASK = EXISTING_MASK

STRENGTH = 1     #@param{'type':'number'}
THRESHOLDS = 0.3    #@param{'type':'number'}
DILATE = 10         #@param{'type':'number'}

FACTOR = 8          #@param{'type':'number'}
FIXED = 0           #@param{'type':'number'}
SEED = 42           #@param{'type':'number'}
NOISE = 0           #@param{'type':'number'}
PLMS = 0            #@param{'type':'number'}
ITERATIONS = 1      #@param{'type':'number'}
SCALE = 15         #@param{'type':'number'}
STEPS = 60          #@param{'type':'number'}

In [5]:
#@title Click and Run { display-mode: "form" }

MASK_Original = True  #@param{'type':'boolean'}
import os, sys, time
import torch
import numpy as np
from omegaconf import OmegaConf
from PIL import Image
from einops import rearrange
from pytorch_lightning import seed_everything
from contextlib import nullcontext
from ldm.util import instantiate_from_config
from ldm.models.diffusion.plms import PLMSSampler
from ldm.models.diffusion.ddim import DDIMSampler
from transformers import logging
from torchvision import transforms

from torchvision.utils import save_image as tv_save_image
from clipseg.models.clipseg import CLIPDensePredT
import cv2

def seed_pre():
    if not FIXED:
        seed_everything(SEED)

def seed_post(device):
    if FIXED:
        seed_everything(SEED)
        return torch.randn([1, 4, HEIGHT // FACTOR, WIDTH // FACTOR], device='cpu').to(torch.device(device.type))
    return None

def load_model(config, ckpt_file=CHECKPOINT,vae_file=VAECKPOINT):
    pl_sd = torch.load(ckpt_file, map_location='cpu')
    sd = pl_sd['state_dict']

    model = instantiate_from_config(config.model)
    model.load_state_dict(sd, strict=False)

    vae_ckpt = torch.load(vae_file, map_location='cpu')
    vae_ignore_keys = {"model_ema.decay", "model_ema.num_updates"}
    vae_dict_1 = {k: v for k, v in vae_ckpt["state_dict"].items() if k[0:4] != "loss" and k not in vae_ignore_keys}
    model.first_stage_model.load_state_dict(vae_dict_1)
    return model

def set_device(model):
    if torch.cuda.is_available():
        device = torch.device('cuda')
        precision = torch.autocast
    else:
        device = torch.device('cpu')
        precision = torch.autocast
    model.to(device.type)
    model.eval()
    return device, precision

def get_img_size(image_file):
    if image_file.endswith(".png"):
      image = Image.open(image_file).convert('RGB')
    else:
      image = Image.open(image_file)
    return image.size

def load_image_mask(device, image_file=IMAGE, mask_file=MASK):
    if image_file.endswith(".png"):
      image = Image.open(image_file).resize((HEIGHT,WIDTH)).convert('RGB')
    else:
      image = Image.open(image_file).resize((HEIGHT,WIDTH))
    w, h = image.size
    w, h = map(lambda x: x - x % 32, (w, h))
    image = image.resize((w, h), resample=Image.LANCZOS)
    image = np.array(image).astype(np.float32) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    image = 2. * image - 1.
    image = image.to(device.type)
    mask = None
    if MASK:
        mask = Image.open(mask_file).convert('L')
        mask = mask.resize((w // FACTOR, h // FACTOR), resample=Image.LANCZOS)
        mask = np.array(mask).astype(np.float32) / 255.0
        mask = np.tile(mask, (4, 1, 1))
        mask = mask[None].transpose(0, 1, 2, 3)
        mask = torch.from_numpy(mask).to(device.type)
    return image, mask

def load_clipseg(weights_file=CLIPSEG):
    clipseg = CLIPDensePredT(version='ViT-B/16', reduce_dim=64, complex_trans_conv=True)
    clipseg.eval()
    clipseg.load_state_dict(torch.load(weights_file, map_location=torch.device('cpu')), strict=False)
    return clipseg

def create_mask(clipseg, image_file=IMAGE, prompt=MASK):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.Resize((WIDTH, HEIGHT)) ])
    if image_file.endswith(".png"):
      image = transform(Image.open(image_file).convert('RGB')).unsqueeze(0)
    else:
      image = transform(Image.open(image_file)).unsqueeze(0)
    size = len(MASK)
    with torch.no_grad():
        preds = clipseg(image.repeat(size,1,1,1), MASK)[0]
    image = torch.sigmoid(preds[0][0])
    for i in range(1, size):
        image += torch.sigmoid(preds[i][0])
    image = (image - image.min()) / image.max()
    if isinstance(THRESHOLDS, list):
        if len(THRESHOLDS) >= 2:
            image = torch.where(image >= THRESHOLDS[1], 1., image)
            image = torch.where(image <= THRESHOLDS[0], 0., image)
        else:
             image = torch.where(image >= THRESHOLDS[0], 1., 0.)
    else:
        image = torch.where(image >= THRESHOLDS, 1., 0.)
    mask_file = os.path.join(FOLDER, f'mask_{time.strftime("%Y%m%d_%H%M%S")}.png')
    tv_save_image(image, mask_file)
    return mask_file

def dilate_mask(mask_file):
    image = cv2.imread(mask_file, 0)
    kernel  = np.ones((DILATE, DILATE), np.uint8)
    dilated = cv2.dilate(image, kernel, iterations=1)
    cv2.imwrite(mask_file, dilated)
    return mask_file

def setup_sampler(model):
    global NOISE
    if IMAGE:
        sampler = DDIMSampler(model)
        sampler.make_schedule(ddim_num_steps=STEPS, ddim_eta=NOISE, verbose=False)
        image, mask = load_image_mask(model.device, IMAGE, MASK)
        sampler.t_enc = int(STRENGTH * STEPS)
        sampler.x0 = model.get_first_stage_encoding(model.encode_first_stage(image))
        sampler.z_enc = sampler.stochastic_encode(sampler.x0, torch.tensor([sampler.t_enc]).to(model.device.type))
        sampler.z_mask = mask

        if MASK and not MASK_Original:
            random = torch.randn(mask.shape, device=model.device)
            sampler.z_enc = (mask * random) + ((1-mask) * sampler.z_enc)
    elif PLMS:
        sampler = PLMSSampler(model)
        NOISE = 0
    else:
        sampler = DDIMSampler(model)
    return sampler

def get_prompts():
    global NEGATIVES
    if NEGATIVES is None:
        NEGATIVES = [''] * len(PROMPTS)
    else:
        NEGATIVES.extend([''] * (len(PROMPTS)-len(NEGATIVES)))
    return zip(PROMPTS, NEGATIVES)

def generate_samples(model, sampler, prompt, negative, start):
    uncond = model.get_learned_conditioning(negative) if SCALE != 1.0 else None
    cond = model.get_learned_conditioning(prompt)
    if IMAGE:
        samples = sampler.inpaint_decode(sampler.z_enc, cond, sampler.t_enc, 
            unconditional_guidance_scale=SCALE, unconditional_conditioning=uncond, 
            z_mask=sampler.z_mask, x0=sampler.x0)
    else:
        shape = [4, HEIGHT // FACTOR, WIDTH // FACTOR]
        samples, _ = sampler.sample(S=STEPS, conditioning=cond, batch_size=1,
            shape=shape, verbose=False, unconditional_guidance_scale=SCALE, 
            unconditional_conditioning=uncond, eta=NOISE, x_T=start)
    x_samples = model.decode_first_stage(samples)
    x_samples = torch.clamp((x_samples + 1.0) / 2.0, min=0.0, max=1.0)
    return x_samples

def save_image(image,ori_w,ori_h):
    name = f'{time.strftime("%Y%m%d_%H%M%S")}.png'
    image = 255. * rearrange(image.cpu().numpy(), 'c h w -> h w c')
    img = Image.fromarray(image.astype(np.uint8))
    new_h = int(ori_h*(WIDTH/ori_w))
    img = img.resize((WIDTH, new_h), resample=Image.LANCZOS)
    img.save(os.path.join(FOLDER, name))
    return name

def save_history(name, prompt, negative):
    with open(os.path.join(FOLDER, HISTORY), 'a') as history:
        history.write(f'{name} -> {"PLMS" if PLMS else "DDIM"}, Seed={SEED}{" fixed" if FIXED else ""}, Scale={SCALE}, Steps={STEPS}, Noise={NOISE}')
        if IMAGE:
            history.write(f', Image={IMAGE}, Strength={STRENGTH}')
        if MASK:
            history.write(f', Mask={MASK}')
        if len(negative):
            history.write(f'\n + {prompt}\n - {negative}\n')
        else:
            history.write(f'\n + {prompt}\n')

def main():
    global MASK
    print('*** Start prompt editting ')
    logging.set_verbosity_error()
    os.makedirs(FOLDER, exist_ok=True)
    seed_pre()

    if isinstance(MASK, list):
        print(f'*** Loading CLIPSeg and creating mask')
        tic1 = time.time()
        clipseg = load_clipseg()
        MASK = create_mask(clipseg)
        if DILATE:
            MASK = dilate_mask(MASK)
        toc1 = time.time()
        print(f'*** CLIPSeg masking time: {(toc1 - tic1):.2f}s')
    if not PROMPTS:
        sys.exit(0)

    print(f'*** Loading diffusion model')
    tic1 = time.time()
    config = OmegaConf.load(CONFIG)
    model = load_model(config)
    device, precision_scope = set_device(model)
    sampler = setup_sampler(model)
    start_code = seed_post(device)
    toc1 = time.time()
    print(f'*** Diffusion model setup time: {(toc1 - tic1):.2f}s')

    counter = 0
    with torch.no_grad():
        with precision_scope(device.type):
            with model.ema_scope():

                for iteration in range(ITERATIONS):
                    for prompt, negative in get_prompts():
                        print(f'*** Iteration {iteration + 1}: {prompt}')
                        tic2 = time.time()
                        images = generate_samples(model, sampler, prompt, negative, start_code)
                        ori_w, ori_h = get_img_size(IMAGE)
                        for image in images:
                            name = save_image(image, ori_w, ori_h)
                            save_history(name, prompt, negative)
                            print(f'*** Saved image: {name}')

                        counter += len(images)
                        toc2 = time.time()
                        print(f'*** Synthesis time: {(toc2 - tic2):.2f}s')

    print(f'*** Total time: {(toc2 - tic1):.2f}s')
    print(f'*** Saved {counter} image(s) to {FOLDER} folder.')

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print('*** Exit')
    except FileNotFoundError as e:
        print(f'*** {e}')


INFO:lightning_lite.utilities.seed:Global seed set to 42


*** Start prompt editting 
*** Loading diffusion model


/usr/local/lib/python3.8/dist-packages/pytorch_lightning/utilities/distributed.py:258: LightningDeprecationWarning: `pytorch_lightning.utilities.distributed.rank_zero_only` has been deprecated in v1.8.1 and will be removed in v1.10.0. You can import it from `pytorch_lightning.utilities` instead.
  rank_zero_deprecation(


LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 64, 64) = 16384 dimensions.
making attention of type 'vanilla' with 512 in_channels
*** Diffusion model setup time: 47.22s
*** Iteration 1: 
Running DDIM Sampling with 60 timesteps


Decoding image: 100%|██████████| 60/60 [00:36<00:00,  1.64it/s]


*** Saved image: 20230106_084203.png
*** Synthesis time: 38.34s
*** Total time: 85.57s
*** Saved 1 image(s) to outputs3 folder.
